In [3]:
import os
import cv2
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from natsort import natsorted
import shutil
from multiprocessing import Pool

In [9]:
target_folder = '../Term_Project_New_Database'
new_folder = '../Term_Project_New_Small'
os.makedirs(new_folder, exist_ok=True)

In [10]:
folders = [x for x in natsorted(os.listdir(target_folder))]
videos = []
dst_folder = []
dst_folder_name = []
for folder in folders:
    folder_videos = natsorted(os.listdir(os.path.join(target_folder, folder)))
    for video in folder_videos:
        name = video.split('.')[0]
        videos.append(os.path.join(target_folder, folder, video))
        dst_folder_name.append(os.path.join(new_folder, folder))
        dst_folder.append(os.path.join(new_folder, folder, '{}.mp4'.format(name)))

In [11]:
def downsample(video, folder_name, dst):
    os.makedirs(os.path.join(new_folder, folder_name), exist_ok=True)
    os.system(f'ffmpeg -y -i {video} -s 720x480 -filter:v fps=25 -c:v libx264 -c:a copy {dst}')

In [12]:
pool = Pool(processes=10)
pool.starmap(downsample, zip(videos, dst_folder_name, dst_folder))
pool.terminate()
pool.join()

In [2]:
source_folder = '../Term_Project_New_Small'
target_folder = '../Term_Project_Full'

In [14]:
originals = [os.path.join(source_folder, x) for x in natsorted(os.listdir(source_folder))]
videos = [os.path.join(original, x) for original in originals for x in natsorted(os.listdir(original))]
video_names = [x.split('.')[0] for original in originals for x in natsorted(os.listdir(original))]

In [15]:
for name in video_names:
    os.makedirs(os.path.join(target_folder, name, 'full'), exist_ok=True)

In [16]:
len(video_names)

1922

In [17]:
def saveFull(video, name):
    capture = cv2.VideoCapture(video)

    count = 0
    while(capture.isOpened()):
        ret, frame = capture.read()

        if not ret:
            break
        cv2.imwrite(os.path.join(target_folder, name, 'full', '{}.jpg'.format(count)), frame)

        count += 1

    capture.release()
    

In [18]:
pool = Pool(processes=40)
pool.starmap(saveFull, zip(videos, video_names))
pool.terminate()
pool.join()

In [4]:
processed_folders = [os.path.join('../Term_Project_Full', x) for x in natsorted(os.listdir('../Term_Project_Full')) if os.path.isdir(os.path.join('../Term_Project_Full', x))]

4818

In [8]:
unprocessed_folders = processed_folders[4630:]
gpu = [i % 4 for i in range(len(unprocessed_folders))]
len(unprocessed_folders)

188

In [9]:
def trigger_crop_portrait(folder, gpu):
    os.system(f'python audiodvp_utils/crop_portrait.py --data_dir {folder} --dest_size 224 --gpu_ids {gpu}')

In [10]:
pool = Pool(processes=4)
pool.starmap(trigger_crop_portrait, zip(unprocessed_folders, gpu))
pool.terminate()
pool.join()

In [1]:
import os
from audiodvp_utils.util import get_file_list, extract_face_emb
from torchvision import transforms
from tqdm import tqdm
from multiprocessing import Pool

data_path = '../Term_Project_Full'
directory = [x for x in sorted(os.listdir(data_path)) if os.path.isdir(os.path.join(data_path, x))]
transforms_input = transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5141, 0.4074, 0.3588], std=[1.0, 1.0, 1.0])
                                    ])

def extractEmb(directory, gpu):
    image_list = get_file_list(os.path.join(data_path, directory, 'crop'))
    save_path = os.path.join(data_path, directory, 'face_emb.pkl')
    extract_face_emb(image_list, save_path, transforms_input, gpu)

In [2]:
gpu = [i % 4 for i in range(len(directory))]

pool = Pool(processes=4)
pool.starmap(extractEmb, zip(directory, gpu))
pool.terminate()
pool.join()

/home/server24/anaconda3/envs/pix2pix/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/server24/anaconda3/envs/pix2pix/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home/server24/anaconda3/envs/pix2pix/lib/python3.8/site-pac